# Hands-on #2b: Matching digits

We now turn to the second task and build a model that
- receives two images of hand-written digits as input and
- outputs a probability that both images show the same digit:

![](img/match_digits.svg)

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')


## Step 1: Preparing the data

We use the MNIST dataset again:

In [ ]:
import tensorflow_datasets as tdfs

tdfs.disable_progress_bar()

mnist_train = tdfs.load(name='mnist', split='train')
mnist_test = tdfs.load(name='mnist', split='test')

mnist_train, mnist_test

We now take pairs of subsequent samples, scale the images as before, and check whether the labels coincide:

In [ ]:
mnist_train.batch(2)

In [ ]:
def match_pairs(samples):
    images, digits = samples['image'], samples['label']
    matching = 1. if digits[0] == digits[1] else 0.
    return (images[0] / 255, images[1]/255), matching


Xy_train = mnist_train.batch(2).map(match_pairs)
Xy_test = mnist_test.batch(2).map(match_pairs)

Xy_train

Let us see how many matching samples we have in our training set:


In [ ]:
Xy_train.reduce(tf.constant((0,0)), lambda count, sample: count + (sample[1], 1))

## Step 2: Building the model

The `Sequential` class allows us to conveniently construct a neural network by stacking layers.

But if we need more flexibility, for example, to construct

- a model with multiple inputs or multiple outputs, or
- a general directed acyclic graph of layers,

we need to use the `tf.keras.Model` class, also known as the functional API of keras.

The idea for our model is that we

1. apply our pre-trained digit-classifier to both images,
2. obtain two probability distributions $p^{(1)}$ and $p^{(2)}$,
3. and use a dense layer to deduce the desired probability:

![](img/digits_matcher.svg)


In [ ]:
CLASSIFIFER_PATH = 'classifier'

classifier = tf.keras.models.load_model(CLASSIFIFER_PATH)
classifier.trainable = False

def build_matcher_dense():
    image_1 = tf.keras.layers.Input((28,28,1))
    image_2 = tf.keras.layers.Input((28,28,1))
    probs_1 = classifier(image_1)
    probs_2 = classifier(image_2)
    both_probs = tf.keras.layers.Concatenate()([probs_1, probs_2])
    dense = tf.keras.layers.Dense(32, activation='relu')(both_probs)
    prediction = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
    matcher = tf.keras.Model(inputs=[image_1, image_2], outputs=[prediction])
    return matcher


Let's train our matcher!

In [ ]:
def train(model, nr_batches=400, nr_epochs=5):
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    history = model.fit(Xy_train.repeat().batch(32).take(nr_batches),
                        validation_data=Xy_test.repeat().batch(32).take(nr_batches // 2),
                        epochs=nr_epochs)

matcher = build_matcher_dense()
train(matcher)

Let us now evaluate the matcher:

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def evaluate(model, nr_samples=1000):
    y_true = np.stack(list(Xy_test.map(lambda _, p: p).take(nr_samples)))
    y_pred = np.round(np.concatenate(model.predict(Xy_test.batch(nr_samples).take(1))))
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

evaluate(matcher)

## Step 3: Building a  model that need not learn

We now replace the last dense classification layer with a lambda layer that need not be trained, using the following observation:

Given the two probability distributions $p^{(1)}$ and $p^{(2)}$, the probability that both digits coincide is $\sum_i p^{(1)}_i p^{(2)}_i$.

We can implement the formula in 3. using a `Lambda` layer. The catch is that all
tensors flowing through the neural network are *batches* of data:

In [ ]:
p1_batch = tf.constant([[0.1, 0.9], [0.5, 0.5]])
p2_batch = tf.constant([[0.2, 0.8],  [0.4, 0.6]])

def compute_prob_equality(p1_batch, p2_batch):
    return tf.reduce_sum(p1_batch * p2_batch, axis=-1)


compute_prob_equality(p1_batch, p2_batch)

In [ ]:
def build_matcher_lambda():
    image_1 = tf.keras.layers.Input((28,28,1))
    image_2 = tf.keras.layers.Input((28,28,1))
    probs_1 = classifier(image_1)
    probs_2 = classifier(image_2)
    # both_probs = tf.keras.layers.Concatenate()([probs_1, probs_2])
    prediction = tf.keras.layers.Lambda(lambda p: tf.reduce_sum(p[0] * p[1], axis=-1, keepdims=True))([probs_1, probs_2])
    matcher = tf.keras.Model(inputs=[image_1, image_2], outputs=[prediction])
    return matcher

Let's see how this model performs!

In [ ]:
matcher = build_matcher_lambda()
evaluate(matcher)


This is quite good, isn't it?